In [30]:
# get all of the json files in proposals_out_2023_07_21
# read them all as one big dataframe

import pandas as pd
import glob

# get all of the json files in proposals_out_2023_07_21
file_names = glob.glob('proposals_out_2023_07_21/*.json')
len(file_names)

122

In [31]:
# monkeypatch json
# monkeypatch using standard python json module

import json

pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

# normalising (unnesting) at the same time (for nested jsons)
pd.io.json._json.loads = lambda s, *a, **kw: pd.json_normalize(simplejson.loads(s))

In [32]:
# read them all as one big dataframe
df = pd.concat([pd.read_json(f) for f in file_names], ignore_index=True)
df.shape

(115562, 7)

In [33]:
df.head()

,id,author,created,votes,title,body,space.id
0,0x06dd5a396a920738b460a99a05acc02c556a1e77c95c...,0xBEBD9051aA63ED001e3Ce3b765FDB5dB2472023D,1637680546,0,超级赛亚人那种形态最牛逼,超级赛亚人那种形态最牛逼,zyx666.eth
1,0xb2838d4e9a47ffaed53646fce891ca48116b57b49734...,0x6aCc1f67290e711912eDDCc4511835C15C252828,1637679181,0,大家多多加入我的空间,,360buy.eth
2,0xf33f99b79bce31a45225ca9af07be6e0e78a279d3fea...,0xE78F896cE38833a6aE75E7BDe08508563802a462,1637668201,12,Do you play Metaverse game?,,emameimei.eth
3,0xf2ce0a63f9cb175c0332849b21ae2056ab7c74295233...,0xF7481bc4581eC74615A8c4604f7bDD7E8cc5e797,1637664349,0,New data interface,New data interface\nCan be provided to strateg...,mtedao.eth
4,0x400629710b0f1c98a19eaf94588382aa3a34bb25dc93...,0xE08DaAB9Be621f0121E9083c45411bB8C3EC63f8,1637663497,0,Do you love ETH,,28828.eth


In [34]:
df.iloc[40]

id          0x5c8fb59f86cd32a4aec9c8c50ed0bb4c303c3a012298...
author             0xDB94448AcEd6415FadaA00Dd313Bde5d37D26fc6
created                                            1637513074
votes                                                       0
title       Is Ethereum really an open and decentralized s...
body                                                         
space.id                                         dilireba.eth
Name: 40, dtype: object

In [35]:
df.columns

Index(['id', 'author', 'created', 'votes', 'title', 'body', 'space.id'], dtype='object')

In [36]:
# * platform
df['platform'] = 'snapshot'
# * rename space.id to platform_deployment_id
df.rename(columns={'space.id': 'platform_deployment_id'}, inplace=True)
# * rename id to proposal_id
df.rename(columns={'id': 'proposal_id'}, inplace=True)
# * author no chg
# * rename created to date
df.rename(columns={'created': 'date'}, inplace=True)
# * votes to votes_count
df.rename(columns={'votes': 'votes_count'}, inplace=True)
# * body to description
df.rename(columns={'body': 'description'}, inplace=True)

# keep only these cols
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'author', 'date', 'votes_count', 'title', 'description']]
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description
0,snapshot,zyx666.eth,0x06dd5a396a920738b460a99a05acc02c556a1e77c95c...,0xBEBD9051aA63ED001e3Ce3b765FDB5dB2472023D,1637680546,0,超级赛亚人那种形态最牛逼,超级赛亚人那种形态最牛逼
1,snapshot,360buy.eth,0xb2838d4e9a47ffaed53646fce891ca48116b57b49734...,0x6aCc1f67290e711912eDDCc4511835C15C252828,1637679181,0,大家多多加入我的空间,
2,snapshot,emameimei.eth,0xf33f99b79bce31a45225ca9af07be6e0e78a279d3fea...,0xE78F896cE38833a6aE75E7BDe08508563802a462,1637668201,12,Do you play Metaverse game?,
3,snapshot,mtedao.eth,0xf2ce0a63f9cb175c0332849b21ae2056ab7c74295233...,0xF7481bc4581eC74615A8c4604f7bDD7E8cc5e797,1637664349,0,New data interface,New data interface\nCan be provided to strateg...
4,snapshot,28828.eth,0x400629710b0f1c98a19eaf94588382aa3a34bb25dc93...,0xE08DaAB9Be621f0121E9083c45411bB8C3EC63f8,1637663497,0,Do you love ETH,


In [37]:
# read date as pd date, it's a seconds epoch timestamp
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,platform,platform_deployment_id,proposal_id,author,date,votes_count,title,description
0,snapshot,zyx666.eth,0x06dd5a396a920738b460a99a05acc02c556a1e77c95c...,0xBEBD9051aA63ED001e3Ce3b765FDB5dB2472023D,2021-11-23 15:15:46,0,超级赛亚人那种形态最牛逼,超级赛亚人那种形态最牛逼
1,snapshot,360buy.eth,0xb2838d4e9a47ffaed53646fce891ca48116b57b49734...,0x6aCc1f67290e711912eDDCc4511835C15C252828,2021-11-23 14:53:01,0,大家多多加入我的空间,
2,snapshot,emameimei.eth,0xf33f99b79bce31a45225ca9af07be6e0e78a279d3fea...,0xE78F896cE38833a6aE75E7BDe08508563802a462,2021-11-23 11:50:01,12,Do you play Metaverse game?,
3,snapshot,mtedao.eth,0xf2ce0a63f9cb175c0332849b21ae2056ab7c74295233...,0xF7481bc4581eC74615A8c4604f7bDD7E8cc5e797,2021-11-23 10:45:49,0,New data interface,New data interface\nCan be provided to strateg...
4,snapshot,28828.eth,0x400629710b0f1c98a19eaf94588382aa3a34bb25dc93...,0xE08DaAB9Be621f0121E9083c45411bB8C3EC63f8,2021-11-23 10:31:37,0,Do you love ETH,


In [38]:
# save
df.to_csv('snapshot_proposals.csv', index=False)